# Process images ready for InstaGAN training

* This notebook takes input in the following format:

1) Cityscapes dataset

`cityscapes-dataset/gtFine_trainvaltest/gtFine/train/train/<city>/<mask>` - containing the fine grain masks of cities in Germany
`cityscapes-dataset/leftImg8bit_trainvaltest/leftImg8bit/train/<city>/<image>` - containing the correspondongimages of the city
    
1) Detroit video frames (target dataset)

`cityscapes/sampleB/<image>` - containing the frames from the video
`cityscapes/sampleB_seg/<mask>` - segmented masks for each of the buildings (generated from Mask_RCNN)
    
* It will process images in the following way:

Move into the correct folder structure for InstaGan (see below)
For the target dataset, remove any images where there is no identified buildings (and hence a blank mask)
Rename images to be of the format ```nnnn.jpg``` and `nnnn_m.jpg` (where n is 0-9 representing the image ID, m is 0-9 representing the mask ID)

* Output structure

Training data will be stored in the following directory structure:

`cityscapes/instagan/train/sampleA/<image>`
`cityscapes/instagan/train/sampleA_seg/<mask>`
`cityscapes/instagan/train/sampleB/<image>`
`cityscapes/instagan/train/sampleB_seg/<mask>`

Images that have no mask in the target dataset will be stored in:

`cityscapes/instagan/no_mask/sampleB/<image>`


In [2]:
import os
from pathlib import Path
import numpy as np
import json
import skimage

In [3]:
BASE_DIR = '../../datasets'
CITYSCAPES_DATASET_BASE = os.path.join(BASE_DIR, 'cityscapes-dataset')
CITYSCAPES_IMAGES = os.path.join(CITYSCAPES_DATASET_BASE, 'leftImg8bit_trainvaltest/leftImg8bit/train')
CITYSCAPES_MASKS = os.path.join(CITYSCAPES_DATASET_BASE, 'gtFine_trainvaltest/gtFine/train')

DERELICT_DATASET_BASE = os.path.join(BASE_DIR, 'cityscapes')
DERELICT_IMAGES = os.path.join(DERELICT_DATASET_BASE, 'sampleB')
DERELICT_MASKS = os.path.join(DERELICT_DATASET_BASE, 'sampleB_seg')

TRAIN_DATASET_BASE = os.path.join(BASE_DIR, 'aurora/instagan')
TRAIN_IMAGES_A = os.path.join(TRAIN_DATASET_BASE, 'train/sampleA')
TRAIN_MASKS_A = os.path.join(TRAIN_DATASET_BASE, 'train/sampleA_seg')
TRAIN_IMAGES_B = os.path.join(TRAIN_DATASET_BASE, 'train/sampleB')
TRAIN_MASKS_B = os.path.join(TRAIN_DATASET_BASE, 'train/sampleB_seg')

NO_MASK_A_DATASET = os.path.join(TRAIN_DATASET_BASE, 'no_mask/sampleA')
NO_MASK_B_DATASET = os.path.join(TRAIN_DATASET_BASE, 'no_mask/sampleB')


* Check target directories exist, if not create them

In [4]:
for directory in [TRAIN_IMAGES_A, TRAIN_MASKS_A, TRAIN_IMAGES_B, TRAIN_MASKS_B, NO_MASK_A_DATASET, NO_MASK_B_DATASET]:
    print('Checking directory {} exists...'.format(directory))
    if not Path(directory).is_dir():
        try:
            os.makedirs(directory)
        except OSError:
            print ("...Failed to create directory!!")
        else:
            print ('...Created OK!')

Checking directory ../../datasets/aurora/instagan/train/sampleA exists...
Checking directory ../../datasets/aurora/instagan/train/sampleA_seg exists...
Checking directory ../../datasets/aurora/instagan/train/sampleB exists...
Checking directory ../../datasets/aurora/instagan/train/sampleB_seg exists...
Checking directory ../../datasets/aurora/instagan/no_mask/sampleA exists...
Checking directory ../../datasets/aurora/instagan/no_mask/sampleB exists...


* Start with Cityscapes Dataset

Move from subdirectory per city to one image and one mask directory with files numbering 0000.jpg, etc

In [5]:
import matplotlib.pyplot as plt
from matplotlib import patches,  lines
from matplotlib.patches import Polygon

def display_images(images, titles=None, cols=4, cmap=None, norm=None,
                   interpolation=None):
    """Display the given set of images, optionally with titles.
    images: list or array of image tensors in HWC format.
    titles: optional. A list of titles to display with each image.
    cols: number of images per row
    cmap: Optional. Color map to use. For example, "Blues".
    norm: Optional. A Normalize instance to map values to colors.
    interpolation: Optional. Image interpolation to use for display.
    """
    titles = titles if titles is not None else [""] * len(images)
    rows = len(images) // cols + 1
    plt.figure(figsize=(14, 14 * rows // cols))
    i = 1
    for image, title in zip(images, titles):
        plt.subplot(rows, cols, i)
        plt.title(title, fontsize=9)
        plt.axis('off')
        plt.imshow(image.astype(np.uint8), cmap=cmap,
                   norm=norm, interpolation=interpolation)
        i += 1
    plt.show()

def display_top_masks(image, mask, class_ids, class_names=['bg', 'building'], limit=4):
    """Display the given image and the top few class masks."""
    to_display = []
    titles = []
    to_display.append(image)
    titles.append("H x W={}x{}".format(image.shape[0], image.shape[1]))
    # Pick top prominent classes in this image
    unique_class_ids = np.unique(class_ids)
    mask_area = [np.sum(mask[:, :, np.where(class_ids == i)[0]])
                 for i in unique_class_ids]
    top_ids = [v[0] for v in sorted(zip(unique_class_ids, mask_area),
                                    key=lambda r: r[1], reverse=True) if v[1] > 0]
    # Generate images and titles
    for i in range(limit):
        class_id = top_ids[i] if i < len(top_ids) else -1
        # Pull masks of instances belonging to the same class.
        m = mask[:, :, np.where(class_ids == class_id)[0]]
        m = np.sum(m * np.arange(1, m.shape[-1] + 1), -1)
        to_display.append(m)
        titles.append(class_names[class_id] if class_id != -1 else "-")
    display_images(to_display, titles=titles, cols=limit + 1, cmap="Blues_r")

In [6]:
def load_image(image_path):
    """Load the specified image and return a [H,W,3] Numpy array.
    """
    # Load image
    image = skimage.io.imread(image_path)
    # If grayscale. Convert to RGB for consistency.
    if image.ndim != 3:
        image = skimage.color.gray2rgb(image)
    # If has an alpha channel, remove it for consistency
    if image.shape[-1] == 4:
        image = image[..., :3]
    return image
    


In [19]:
image_id = 0
no_mask_count = 0
image_list = Path(DERELICT_IMAGES).glob('**/*.png')
for image_path in image_list:
    image_file = os.path.basename(image_path)
    print('Processing image {} / {}'.format(image_id, image_path))
    image = load_image(image_path)
    image_out_path = os.path.join(TRAIN_IMAGES_B, '{}.png'.format(image_id))
    # print('Saving image as {}'.format(image_out_path))
    image_file_no_ext = image_file.split('.')[0]
    mask_files = Path(DERELICT_MASKS).glob('**/{}_*.png'.format(image_file_no_ext))
    mask_id = 0
    for i, mask_file in enumerate(mask_files):
        print(mask_file)
        mask = skimage.io.imread(mask_file)
        if np.max(mask) != 0:
            mask_out_path = os.path.join(TRAIN_MASKS_B, '{}_{}.png'.format(image_id, mask_id))
            skimage.io.imsave(mask_out_path, mask)
            mask_id += 1
        else:
            print('...No mask')
            no_mask_count += 1
            image_out_path = os.path.join(NO_MASK_B_DATASET, '{}.png'.format(image_id))
    skimage.io.imsave(image_out_path, image)
    image_id += 1

Processing image 0 / ../../datasets/cityscapes/sampleB/out1450.png
../../datasets/cityscapes/sampleB_seg/out1450_0.png
Processing image 1 / ../../datasets/cityscapes/sampleB/out816.png
../../datasets/cityscapes/sampleB_seg/out816_0.png
...No mask
Processing image 2 / ../../datasets/cityscapes/sampleB/out4528.png
../../datasets/cityscapes/sampleB_seg/out4528_0.png
Processing image 3 / ../../datasets/cityscapes/sampleB/out3247.png
../../datasets/cityscapes/sampleB_seg/out3247_0.png
Processing image 4 / ../../datasets/cityscapes/sampleB/out2159.png
../../datasets/cityscapes/sampleB_seg/out2159_0.png
Processing image 5 / ../../datasets/cityscapes/sampleB/out5636.png
../../datasets/cityscapes/sampleB_seg/out5636_0.png
...No mask
Processing image 6 / ../../datasets/cityscapes/sampleB/out5150.png
Processing image 7 / ../../datasets/cityscapes/sampleB/out3521.png
../../datasets/cityscapes/sampleB_seg/out3521_0.png
Processing image 8 / ../../datasets/cityscapes/sampleB/out1336.png
../../dataset

Processing image 65 / ../../datasets/cityscapes/sampleB/out5346.png
../../datasets/cityscapes/sampleB_seg/out5346_0.png
../../datasets/cityscapes/sampleB_seg/out5346_1.png
Processing image 66 / ../../datasets/cityscapes/sampleB/out1120.png
../../datasets/cityscapes/sampleB_seg/out1120_0.png
Processing image 67 / ../../datasets/cityscapes/sampleB/out4070.png
../../datasets/cityscapes/sampleB_seg/out4070_0.png
...No mask
Processing image 68 / ../../datasets/cityscapes/sampleB/out2401.png
Processing image 69 / ../../datasets/cityscapes/sampleB/out396.png
../../datasets/cityscapes/sampleB_seg/out396_0.png
../../datasets/cityscapes/sampleB_seg/out396_1.png
Processing image 70 / ../../datasets/cityscapes/sampleB/out1108.png
../../datasets/cityscapes/sampleB_seg/out1108_1.png
../../datasets/cityscapes/sampleB_seg/out1108_0.png
Processing image 71 / ../../datasets/cityscapes/sampleB/out6101.png
../../datasets/cityscapes/sampleB_seg/out6101_0.png
...No mask
Processing image 72 / ../../datasets/

KeyboardInterrupt: 

In [ ]:
print('Stats:')
print('Images processed: {}'.format(image_id))
print('Images with no masks: {}'.format(no_mask_count))